## Scale out to distributed Dask on CPU

**When to use**: 
- When data becomes **too large** to read in to local memory
- Need distributed clusters to handle the compute end to end
- Read data directly into remote Dask workers to overcome the bottleneck of local memory

**How to use**:
- `nc.initialize_cluster()` to intialize a remote distributed Dask cluster with CPU or GPU
- `cluster.close()` to shutdown the remote cluster after computation

In [2]:
from hyperplane import notebook_common as nc

client, cluster = nc.initialize_cluster(num_workers = 2)

👉 Hyperplane: selecting worker node pool
👉 Hyperplane: selecting scheduler node pool
Creating scheduler pod on cluster. This may take some time.
👉 Hyperplane: spinning up a dask cluster with a scheduler as a standalone container.
👉 Hyperplane: In a few minutes you'll be able to access the dashboard at https://shakdemo.hyperplane.dev/dask-cluster-f6d102b7-4881-40c1-8288-7eb5b7a54856/status
👉 Hyperplane: to get logs from all workers, do `cluster.get_logs()`


In [3]:
client

Client Scheduler: tcp://dask-cluster-f6d102b7-4881-40c1-8288-7eb5b7a54856.jhub-67l2bgl0:8786 Dashboard: http://dask-cluster-f6d102b7-4881-40c1-8288-7eb5b7a54856.jhub-67l2bgl0:8787/status,Cluster Workers: 30 Cores: 30 Memory: 22.35 GiB


## Example 1 K-means

In [5]:
import dask.dataframe as dd 
data_cpu =dd.read_csv("gs://shakdemo-hyperplane/data/synthetic_data/*.csv")
data_cpu.head(2)

,x,y,label
0,0.149094,5.599997,0
1,4.918161,-0.026502,5


In [6]:
%%time
from dask_ml.cluster import KMeans as daskKMeans

# Setup the Dask task graph.
# Instantiate, train and predict.
kmeans_dask = daskKMeans(init="k-means||",
                         n_clusters=6,
                         random_state=0)
kmeans_dask.fit(data_cpu[['x','y']])
kmeans_dask_df = kmeans_dask.predict(data_cpu[['x','y']])

# Execute the Dask task graph.
labels_dask = kmeans_dask_df.compute()

CPU times: user 4.71 s, sys: 318 ms, total: 5.03 s
Wall time: 3min 11s


In [8]:
labels_dask.shape

(24000000,)

In [9]:
cluster.close()

## Example 2 - lightgbm